In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

In [3]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"]=GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## Predefine tools in the previous ipynb

## Custom Tool

In [8]:
from langchain.agents import tool
@tool
def get_word_length(word:str)->int:
    """Return the word length"""
    return len(word)

In [9]:
get_word_length.invoke("Gyanbardhan")

11

In [11]:
@tool
def multiply(a:int,b:int)->int:
    "Multiply two numbers"
    return a*b

In [14]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [16]:
multiply.invoke({'a':10,'b':20})

200

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [18]:
search=TavilySearchResults()

In [19]:
tools=[search]

In [20]:
from langchain import hub

In [21]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [23]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

agent_scratchpad: basically a structure in which agent should work


In [24]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,tools,prompt)

In [25]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [30]:
agent_executor.invoke({"input":"Weather of Bangalore"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Weather in Bangalore'}`


[{'title': 'Weather in Bangalore', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bangalore', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1745814602, 'localtime': '2025-04-28 10:00'}, 'current': {'last_updated_epoch': 1745813700, 'last_updated': '2025-04-28 09:45', 'temp_c': 27.4, 'temp_f': 81.3, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 141, 'wind_dir': 'SE', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 27.9, 'feelslike_f': 82.1, 'windchill_c': 28.4, 'windchill_f': 83.1, 'heatindex_c': 29.0, 'heatindex_f': 84.2, 'dewpoint_c': 16.3, 'dewpoint_f': 61.3, 'vis_km': 6.0

{'input': 'Weather of Bangalore', 'output': '\n'}